In [1]:
source = "Recommendations";

In [2]:
using NBInclude
@nbinclude("../ProductionAlphas/Alpha.ipynb");

In [18]:
ENV["COLUMNS"] = 999999;
ENV["LINES"] = 50;
recommend_related_series = false;

In [19]:
anime = DataFrame(CSV.File("../../data/raw_data/anime.csv", ntasks = 1))
anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"))
anime_to_uid = innerjoin(anime_to_uid, anime, on = "anime_id");

In [60]:
predictions = read_recommendee_alpha("CombineSignals").rating
rating_df = DataFrame(
    "uid" => 0:length(predictions)-1,
    "rating" => predictions,
    "alpha" => predictions - read_recommendee_alpha("UserItemBiases").rating,
    "std" => read_recommendee_alpha("PredictedErrors").rating,
    "p" => read_recommendee_alpha("GNN_implicit").rating,
);
# penalize items for uncertainty
rating_df.score = rating_df.rating - 1 * rating_df.std; # TODO optimize this

In [61]:
# evaluate our insample predictions
df = get_recommendee_list()
@debug rmse(df.rating, predictions[df.item])
@debug mae(df.rating, predictions[df.item])
@debug r2(df.rating, predictions[df.item])

[ Debug: 20220225 04:41:59 1.3023361
[ Debug: 20220225 04:41:59 0.9796402
[ Debug: 20220225 04:41:59 0.43928957


In [62]:
function dont_recommend!(df, mask)
    df.rating[mask] .= 0
    df.alpha[mask] .= -Inf
    df.score[mask] .= -Inf
    df.p[mask] .= 0
end;

# don't recommend shows that the user has already seen before
dont_recommend!(rating_df, get_recommendee_list().item)
dont_recommend!(rating_df, get_implicit_list().item)

# don't recommend shows related to shows they have seen before
if !recommend_related_series
    related_series =
        get_alpha("ItemCFRelated.strict_relations", "recommendee_inference").rating .!= 0
    dont_recommend!(rating_df, related_series)
end;

# renormalize p
rating_df.p = rating_df.p ./ sum(rating_df.p);

In [63]:
rec_df = innerjoin(anime_to_uid, rating_df, on = "uid");
rec_df = rec_df[
    :,
    [
        "anime_id",
        "uid",
        "title",
        "genres",
        "medium",
        "rating",
        "alpha",
        "std",
        "score",
        "p",
        "nsfw",
    ],
];

In [64]:
length(get_recommendee_list().item) + length(get_implicit_list().item)

845

In [65]:
median(rec_df.p), quantile(rec_df.p, 0.75), quantile(rec_df.p, 0.9)

(8.9811516f-7, 1.852338573371526e-5, 0.00011727028904715533)

In [112]:
# the model predicts ratings conditional on the user watching the series
# we want to predict ratings for items the user has not already decided to watch
# to mitigate the sampling bias, we first filter to series that the user is likely to be interested in
min_p = quantile(filter(x -> x.medium == "tv", rec_df).p, 0.9);
min_alpha =
    quantile(filter(x -> x.p > min_p, filter(x -> x.medium == "tv", rec_df)).alpha, 0.75)

0.16658258438110352

In [113]:
function sampling_filter(df)
    filter(x -> x.p >= min_p, filter(x -> x.alpha .> min_alpha, df))
end;

In [114]:
function pretty_display(df)
    df = sort(df, :score, rev = true)
    select(df, Not(:score))
end;

In [111]:
filter(x -> x.medium == "tv", rec_df |> sampling_filter) |> pretty_display

,anime_id,uid,title,genres,medium,rating,alpha,std,p,nsfw
,Int64,Int64,String,String?,String7,Float32,Float32,Float32,Float32,String7
1,31988,11173,Hibike! Euphonium 2,"['Drama', 'Music', 'School']",tv,7.43699,0.411153,0.799064,0.00168306,white
2,5941,4377,Cross Game,"['Comedy', 'Drama', 'Romance', 'School', 'Shounen', 'Sports']",tv,7.38577,0.305717,0.976942,0.00104641,white
3,27989,9666,Hibike! Euphonium,"['Drama', 'Music', 'School']",tv,7.26322,0.602006,0.94387,0.00258483,white
4,1065,967,Touch,"['Drama', 'Romance', 'School', 'Shounen', 'Slice of Life', 'Sports']",tv,7.26128,0.379541,0.964011,0.00076882,white
5,44235,17740,Pui Pui Molcar,"['Cars', 'Comedy', 'Kids']",tv,7.3666,0.299013,1.07456,0.00034708,white
6,48736,18401,Sono Bisque Doll wa Koi wo Suru,"['Romance', 'School', 'Seinen', 'Slice of Life']",tv,6.95735,0.196341,0.702034,0.000317434,white
7,39783,15711,5-toubun no Hanayome ∬,"['Comedy', 'Harem', 'Romance', 'School', 'Shounen']",tv,6.87952,0.299643,0.646631,0.000263215,white
8,39570,15596,High Score Girl II,"['Comedy', 'Game', 'Romance', 'School', 'Seinen']",tv,6.97137,0.276316,0.756231,0.000592566,white
9,3750,3248,Maria-sama ga Miteru 4th,"['Girls Love', 'School', 'Shoujo', 'Slice of Life']",tv,6.92144,0.535401,0.803822,0.000260241,white


In [103]:
filter(x -> x.medium == "tv", rec_df |> sampling_filter) |> pretty_display

,anime_id,uid,title,genres,medium,rating,alpha,std,p,nsfw
,Int64,Int64,String,String?,String7,Float32,Float32,Float32,Float32,String7
1,31988,11173,Hibike! Euphonium 2,"['Drama', 'Music', 'School']",tv,7.43699,0.411153,0.799064,0.00168306,white
2,5941,4377,Cross Game,"['Comedy', 'Drama', 'Romance', 'School', 'Shounen', 'Sports']",tv,7.38577,0.305717,0.976942,0.00104641,white
3,27989,9666,Hibike! Euphonium,"['Drama', 'Music', 'School']",tv,7.26322,0.602006,0.94387,0.00258483,white
4,1065,967,Touch,"['Drama', 'Romance', 'School', 'Shounen', 'Slice of Life', 'Sports']",tv,7.26128,0.379541,0.964011,0.00076882,white
5,25835,9373,Shirobako,"['Comedy', 'Drama']",tv,7.11743,0.138514,0.967802,0.00261632,white
6,1530,1388,Kanon (2006),"['Drama', 'Romance', 'Slice of Life', 'Supernatural']",tv,7.13544,0.692509,1.04339,0.00227492,white
7,488,458,Ichigo Mashimaro,"['Comedy', 'Slice of Life']",tv,6.9406,0.731873,0.855252,0.00104676,white
8,30831,10655,Kono Subarashii Sekai ni Shukufuku wo!,"['Adventure', 'Comedy', 'Fantasy', 'Parody', 'Supernatural']",tv,6.77358,0.177849,0.709692,0.00220403,white
9,135,114,Hikaru no Go,"['Comedy', 'Game', 'Shounen', 'Supernatural']",tv,6.87152,0.275471,0.851506,0.000676344,white


In [45]:
filter(x -> x.medium == "tv", rec_df |> sampling_filter) |> pretty_display

,anime_id,uid,title,genres,medium,rating,alpha,std,p,nsfw
,Int64,Int64,String,String?,String7,Float32,Float32,Float32,Float32,String7
1,8861,5539,"Yosuga no Sora: In Solitude, Where We Are Least Alone.","['Drama', 'Ecchi', 'Harem', 'Romance']",tv,6.84904,2.40808,2.40458,0.000865995,gray
2,1065,967,Touch,"['Drama', 'Romance', 'School', 'Shounen', 'Slice of Life', 'Sports']",tv,7.47128,0.589532,1.04341,0.0006355,white
3,28891,9877,Haikyuu!! Second Season,"['Comedy', 'Drama', 'School', 'Shounen', 'Sports']",tv,7.66261,0.397465,0.793707,0.0003555,white
4,1530,1388,Kanon (2006),"['Drama', 'Romance', 'Slice of Life', 'Supernatural']",tv,7.28862,0.845688,1.08979,0.00188043,white
5,5941,4377,Cross Game,"['Comedy', 'Drama', 'Romance', 'School', 'Shounen', 'Sports']",tv,7.30743,0.227377,0.953207,0.000864957,white
6,27989,9666,Hibike! Euphonium,"['Drama', 'Music', 'School']",tv,7.31936,0.658142,0.912277,0.0021366,white
7,31988,11173,Hibike! Euphonium 2,"['Drama', 'Music', 'School']",tv,7.45447,0.428632,0.746721,0.00139121,white
8,19111,7999,Love Live! School Idol Project 2nd Season,"['Music', 'School', 'Slice of Life']",tv,7.40361,1.15205,0.638961,0.000705566,white
9,25835,9373,Shirobako,"['Comedy', 'Drama']",tv,7.19743,0.218513,0.825989,0.00216263,white


In [15]:
rec_df |> sampling_filter |> pretty_display

,anime_id,uid,title,genres,medium,rating,alpha,std,p,nsfw
,Int64,Int64,String,String?,String7,Float32,Float32,Float32,Float32,String7
1,11979,6651,Mahou Shoujo Madoka★Magica Movie 2: Eien no Monogatari,"['Drama', 'Psychological', 'Suspense']",movie,9.35505,2.34811,0.450734,0.000966419,white
2,11977,6650,Mahou Shoujo Madoka★Magica Movie 1: Hajimari no Monogatari,"['Drama', 'Psychological', 'Suspense']",movie,9.01349,2.18564,0.431936,0.00107685,white
3,6862,4780,K-On!: Live House!,"['Comedy', 'Music', 'School', 'Slice of Life']",special,7.53794,1.21402,0.583172,0.00116204,white
4,4565,3712,Tengen Toppa Gurren Lagann Movie 2: Lagann-hen,"['Action', 'Mecha', 'Sci-Fi', 'Space', 'Super Power']",movie,7.79975,0.535449,0.903404,0.000628137,white
5,28891,9877,Haikyuu!! Second Season,"['Comedy', 'Drama', 'School', 'Shounen', 'Sports']",tv,7.66261,0.397465,0.793707,0.0003555,white
6,9734,5866,K-On!!: Keikaku!,"['Comedy', 'Music', 'School', 'Slice of Life']",special,7.5951,1.25265,0.769727,0.000705473,white
7,19111,7999,Love Live! School Idol Project 2nd Season,"['Music', 'School', 'Slice of Life']",tv,7.40361,1.15205,0.638961,0.000705566,white
8,31988,11173,Hibike! Euphonium 2,"['Drama', 'Music', 'School']",tv,7.45447,0.428632,0.746721,0.00139121,white
9,34914,12658,New Game!!,"['Comedy', 'Game', 'Slice of Life']",tv,7.11493,0.853506,0.473952,0.000567838,white
